## Creatore dataset piu piccoli per rapidi test

In [22]:
from pathlib import Path

def set_dataset_path(path_str):
    """
    Imposta il path del dataset e restituisce un oggetto Path per il resto del notebook.
    """
    dataset_path = Path(path_str).expanduser().resolve()
    print(f"Percorso dataset impostato a:\n{dataset_path}")
    return dataset_path

```
QUI INSERISCI IL PATH ALLA CARTELLA CHE CONTIENE IL DATASET

In [23]:
dataset_path = set_dataset_path('kaist-cvpr15')  # Modifica questo percorso secondo le tue esigenze 

# Karolina /scratch/project/eu-25-19/kaist-cvpr15

Percorso dataset impostato a:
/home/honey/ADL-Project/kaist-cvpr15


Funzione che crea una sub directory per ospitare il nuovo sub dataset mantenendo disposizione richiesta da ICAFusion

In [24]:
def create_subset_folder_structure(subset_path):
    # Definisci la cartella di output
    # kaist-cvpr15/
    #       Kaist_x_icafusion_v1.0/
    #                   visible/
    #                   infrared/
    #                   labels/
    
    output_root = dataset_path / subset_path

    # Crea le cartelle dentro questa nuova root
    for mode in ["train", "test"]:
        (output_root / "visible" / mode).mkdir(parents=True, exist_ok=True)
        (output_root / "infrared" / mode).mkdir(parents=True, exist_ok=True)
        (output_root / "labels" / mode).mkdir(parents=True, exist_ok=True)

    return output_root
   


Funzione per copiare n sample dal training set e m sample dal test set

In [25]:
from pathlib import Path
import shutil
import math

def crea_subset_sequenziale(output_root, n_train, n_test):
    """
    Crea subset prendendo sample equidistanti dal train e dal test,
    mantenendo la struttura {visible, infrared, labels}/{train, test}/
    """
    def sample_seq(lst, N):
        # Campiona N sample equamente distribuiti dalla lista ordinata
        L = len(lst)
        if N >= L:  # se ne chiedi più di quanti ne hai, usa tutti
            return lst
        step = L / N
        return [lst[math.floor(i*step)] for i in range(N)]


    input_root =  dataset_path / "Kaist_x_icafusion_v1.0" 

    for split, n in [('train', n_train), ('test', n_test)]:
        vis_dir = input_root / "visible" / split
        ir_dir = input_root / "infrared" / split
        lbl_dir = input_root / "labels" / split

        vis_names = {f.stem for f in vis_dir.iterdir() if f.is_file()}
        ir_names  = {f.stem for f in ir_dir.iterdir() if f.is_file()}
        lbl_names = {f.stem for f in lbl_dir.iterdir() if f.is_file()}
        triple = sorted(list(vis_names & ir_names & lbl_names))

        if not triple:
            print(f"Nessun sample disponibile per {split}")
            continue

        if n > len(triple):
            print(f"Richiesti {n} sample per {split}, disponibili solo {len(triple)}. Verranno copiati tutti.")
            sel = triple
        else:
            sel = sample_seq(triple, n)

        # Crea cartelle di output
        for cat in ["visible", "infrared", "labels"]:
            (output_root / cat / split).mkdir(parents=True, exist_ok=True)

        for name in sel:
            for cat, ext in [("visible", ".jpg"), ("infrared", ".jpg"), ("labels", ".txt")]:
                src = input_root / cat / split / f"{name}{ext}"
                dst = output_root / cat / split / f"{name}{ext}"
                if src.is_file():
                    shutil.copy2(src, dst)
                else:
                    print(f"[WARN] File mancante: {src}")

    print(f"\nSubset creato in {output_root}")


Funzione per conformity checks.  
* Conta numero di triplette e ritorna eventuali sample spaiati.  
* Se il numero di triplette e' identico al numero di sample richiesti e non sono presenti elementi spaiati la procedura e' andata a buon fine!

In [31]:
from pathlib import Path

def check_triplette(output_root, split="train"):
    """
    Conta e verifica le triplette (visible + infrared + labels) presenti per una split (train/test).
    """

    visible_dir = output_root / "visible" / split
    infrared_dir = output_root / "infrared" / split
    labels_dir = output_root / "labels" / split

    vis_files = {f.stem for f in visible_dir.iterdir() if f.is_file()}
    ir_files = {f.stem for f in infrared_dir.iterdir() if f.is_file()}
    lbl_files = {f.stem for f in labels_dir.iterdir() if f.is_file()}

    triplette = vis_files & ir_files & lbl_files
    print(f"Totale file in visible/{split}:  {len(vis_files)}")
    print(f"Totale file in infrared/{split}: {len(ir_files)}")
    print(f"Totale file in labels/{split}:   {len(lbl_files)}")

    print(f"\nNumero di triplette complete (presenti in tutte e tre le cartelle): {len(triplette)}")

    # lista sugli "orfani":
    solo_vis = vis_files - triplette
    solo_ir = ir_files - triplette
    solo_lbl = lbl_files - triplette

    if solo_vis:
        print(f"File solo in visible (non triplette): {len(solo_vis)}")
    if solo_ir:
        print(f"File solo in infrared (non triplette): {len(solo_ir)}")
    if solo_lbl:
        print(f"File solo in labels (non triplette): {len(solo_lbl)}")


  

### Struttura Dataset originale recap  
-> 26835 training samples (one visible - one infrared - one label(YOLO))  
-> 45140 testing  samples (one visible - one infrared - one label(YOLO))  

## Riempimenti nuove sub_directory
```
RICHIEDE ESISTENZA DI Kaist_x_icafusion_v1.0 dall'altro ipynb!!

#### ExtraSmall_1000_1000  

> 1000 training sample  
> 1000 testing  sample   

In [33]:
# Creazione della struttura di cartelle per il nuovo subset
output_path_ExtraSmall_1000_1000 = create_subset_folder_structure('Kaist_x_icafusion_vExtraSmall_1000_1000')


# IMPORTANTE RICORDARE NOME VARIABILE ASSEGNATA QUA!

In [34]:
# Riempimento del subset con 1000 immagini di train e 1000 di test
crea_subset_sequenziale(output_path_ExtraSmall_1000_1000, n_train=1000, n_test=1000)


Subset creato in /home/honey/ADL-Project/kaist-cvpr15/Kaist_x_icafusion_vExtraSmall_1000_1000


In [39]:
# Verifica delle triplette nel subset creato
check_triplette(output_path_ExtraSmall_1000_1000, split="train")

# Verifica delle triplette nel subset creato
check_triplette(output_path_ExtraSmall_1000_1000, split="test")

Totale file in visible/train:  1000
Totale file in infrared/train: 1000
Totale file in labels/train:   1000

Numero di triplette complete (presenti in tutte e tre le cartelle): 1000
Totale file in visible/test:  1000
Totale file in infrared/test: 1000
Totale file in labels/test:   1000

Numero di triplette complete (presenti in tutte e tre le cartelle): 1000


#### ExtraSmall_1000_200

> 1000 training sample  
> 200 testing  sample 


In [36]:
output_path_ExtraSmall_1000_200 = create_subset_folder_structure('Kaist_x_icafusion_vExtraSmall_1000_200')
# Riempimento del subset con 1000 immagini di train e 200 di test

In [37]:
crea_subset_sequenziale(output_path_ExtraSmall_1000_200, n_train=1000, n_test=200)


Subset creato in /home/honey/ADL-Project/kaist-cvpr15/Kaist_x_icafusion_vExtraSmall_1000_200


In [40]:
check_triplette(output_path_ExtraSmall_1000_200, split="train")
check_triplette(output_path_ExtraSmall_1000_200, split="test")

Totale file in visible/train:  1000
Totale file in infrared/train: 1000
Totale file in labels/train:   1000

Numero di triplette complete (presenti in tutte e tre le cartelle): 1000
Totale file in visible/test:  200
Totale file in infrared/test: 200
Totale file in labels/test:   200

Numero di triplette complete (presenti in tutte e tre le cartelle): 200
